# BERT Topic Modeling of Survey Measurements of Masculinity

Purpose: Employ BERT, a topic modeling algorithm, to automatically cluster text data into particular topics. In this particular example, I am analyzing the common topics/themes of ~500 survey questions from surveys that measure masculinity. 

Topic Modeling Information: https://maartengr.github.io/BERTopic/index.html

Inspiration: https://github.com/pinecone-io/examples/tree/master/learn/experimental/algos-and-libraries/bertopic

## Imports and Setup

In [74]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%run Functions/Functions.ipynb

In [2]:
df = pd.read_csv('../Data/Questions_Final.csv')

# Comment the following line if evaluating more than the CMNI
df = df[df["Scale"] == "Conformity to Masculine Norms Inventory"]

print(df.shape)
# Put questions in a list
docs = df['Question Text'].to_list()

(94, 7)


## Naive BERTopic (default hyperparameters)

In [ ]:
# Naive vectorizer to remove stop words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model, nr_topics='auto')
# fit model
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_info_df = topic_model.get_topic_info()
topic_info_df

In [ ]:
bar_chart = topic_model.visualize_barchart()

# Update filepath as needed
bar_chart.write_image("../Visualizations/BERTopic_Bar_Chart.png", width=800, height=600)
bar_chart

In [ ]:
# Visualize Topics
# intertopic_map = model.visualize_topics().show()
# heat_map = topic_model.visualize_heatmap()

# # Update filepath as needed to save
# heat_map.write_image("../Visualizations/BERTopic_Heat_Map.png", width=800, height=600)
# heat_map

## Investigate Factor Loadings of Original CMNI vs. BERTopic

In [ ]:
### Get cluster assignments of BERTopic and then merge into dataset
full_BERTopic_info = topic_model.get_document_info(docs)

# Join the "Document" column of full_BERTopic_info with the "Question Text" column of df
merged_df = pd.merge(df, full_BERTopic_info, left_on="Question Text", right_on="Document", how="inner")

# Drop the duplicate "Document" column if needed
merged_df.drop("Document", axis=1, inplace=True)

merged_df_subset = merged_df[["Factor", "Question Text", "Topic", "Name"]]
# Rename columns for readability
merged_df_subset = merged_df_subset.rename(columns={
    "Question Text": "Document",
    "Factor": "Original Factor",
    "Topic": "BERTopic Topic",
    "Name": "BERTopic Name"
})

merged_df_subset = merged_df_subset[['Document', 'Original Factor', 'BERTopic Topic', 'BERTopic Name']]
merged_df_subset.head()

In [ ]:
# Create a cross-tabulation table to count the frequency of occurrences of each combination of "Original Factor" and "BERTopic Topic".
cross_tab = pd.crosstab(merged_df_subset["Original Factor"], merged_df_subset["BERTopic Topic"])

# Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(cross_tab, cmap="Blues", annot=True, fmt="d")
plt.title("Cross-Tabulation of Original Factors and BERTopic Topics")
plt.xlabel("BERTopic Topics")
plt.ylabel("Original Factors")
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("../Visualizations/BERTopic_CrossTab.png")
plt.show()

In [ ]:
# Perform chi-squared test of independence to determine whether there is a stat. sig. assoc. 
# to see whether obs. freqs. in the contingency table differ significantly.
from scipy.stats import chi2_contingency

# Convert text values to categorical variables
merged_df_subset["Original Factor"] = pd.Categorical(merged_df_subset["Original Factor"])
merged_df_subset["BERTopic Topic"] = pd.Categorical(merged_df_subset["BERTopic Topic"])

# Create a contingency table
contingency_table = pd.crosstab(merged_df_subset["Original Factor"], merged_df_subset["BERTopic Topic"])

chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared statistic:", chi2)
print("P-value:", p_value)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

## How does BERTopic work?

**Data -> Sentence Embeddings** (transforming text into machine readable version - usually 768/384 dimensions)


Sentence Embeddings via Transformers Architecture
    Transformers use an encode/decode network. English into a context vector and then decoded. However the context vector is so complex this creates a bottleneck into the decoder. So the attention mechanism tells which decoder parts to focus on. In 2017, attention paper released that found they can remove the recurrent parts of the network and just keep the attention mechanism so high-performing. (called a transformer).

New attention (transformer): 
(1) have positional encoding (sense of order of the tokens)
(2) self-attention (how a word is applied to all other words in the sentence). Rather than embed the meaning of the individual word, it embeds the context of the words around it.
(3) multi-head attention (parallelization)

We can take a pre-trained core (e.g. BERT) and can fine-tune it.

So now we have a vector with 768/384 dimensions.

    
**Embeddings -> UMAP** (dimensionality reduction)

UMAP helps us compress embeddings into a smaller vector small (2 or 3 dimensions).
Understands density of particular areas in the data through k_nearest_neighbors. Understands distance between different vector rows


**UMAP -> clustering with HDBSCAN**
Here we can define the number of points needed to define a cluster. (by default something like 5)

(Number of topics can be determined by how we set parameters like min_cluster_size and min_samples (how dense the core of a cluster needs to be to group))

**clustering with HDBSCAN -> C-TF-IDF**
Looks at freq. of words with a particular cluster, and see how common these words are.


## Customizing BERTopic

Inspiration: https://www.pinecone.io/learn/bertopic/

For information on the meaning of parameters, look up the Functions.ipynb file

In [3]:
docs = df['Question Text'].to_list()

In [17]:
# Defining embedding_model and vectorizer_model outside the run_custom_BERTopic function as they are required for evaluation metrics as well
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Most popular model - Hugging Face: https://huggingface.co/sentence-transformers?sort_models=downloads#models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# remove stopwords/stop phrases containing 1 and 2 words
# NGRAM_RANGE is an important choice
stop_words = list(stopwords.words('english'))
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)


In [49]:
# Run custom model
model = run_custom_BERTopic(docs, embedding_model, vectorizer_model, min_cluster_size=2, min_samples=1)
#model = run_custom_BERTopic(docs, embedding_model, vectorizer_model, min_cluster_size=10, min_samples=10)
topics, probs = model.fit_transform(docs)

topic_info_df = model.get_topic_info()

num_topics = len(topic_info_df) - 1 # DOES NOT INCLUDE OUTLIERS TOPIC. ONLY COHERENT TOPICS
#topic_info_df

# Visualize Topics
# model.visualize_topics()

2024-05-28 01:35:29,515 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-05-28 01:35:29,648 - BERTopic - Embedding - Completed ✓
2024-05-28 01:35:29,648 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-28 01:35:30,132 - BERTopic - Dimensionality - Completed ✓
2024-05-28 01:35:30,133 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-28 01:35:30,140 - BERTopic - Cluster - Completed ✓
2024-05-28 01:35:30,141 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-28 01:35:30,152 - BERTopic - Representation - Completed ✓


## Evaluation Metrics

In [37]:
### GET FULL BERTopic INFO

full_BERTopic_info = model.get_document_info(docs)

# Join the "Document" column of full_BERTopic_info with the "Question Text" column of df
merged_df = pd.merge(df, full_BERTopic_info, left_on="Question Text", right_on="Document", how="inner")

# Drop the duplicate "Document" column if needed
merged_df.drop("Document", axis=1, inplace=True)

merged_df_subset = merged_df[["Factor", "Question Text", "Topic", "Name"]]
# Rename columns for readability
merged_df_subset = merged_df_subset.rename(columns={
    "Question Text": "Document",
    "Factor": "Original Factor",
    "Topic": "BERTopic Topic",
    "Name": "BERTopic Name"
})

In [45]:
### Gather evaluation metrics

# Create an empty DataFrame with columns for the metrics
metrics_df = pd.DataFrame(columns=['Coherence (NPMI)', 'Silhouette Score', 'ARI', 'Purity', 'NMI'])


coh = coherence_score(docs, model, vectorizer_model)

# play with seeing if you actually need topics param here
sil = silhouette_metric(docs, embedding_model, model, topics)

true_values = merged_df_subset['Original Factor']
predicted_values = merged_df_subset['BERTopic Topic'] # NOTE THAT TOPIC -1 is included in the predicted_labels. Should we remove?!?!?!
ari, purity, nmi = evaluation_metrics(true_values, predicted_values)

metrics_df.loc["model 1"] = [coh, sil, ari, purity, nmi]

metrics_df

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Coherence (NPMI),Silhouette Score,ARI,Purity,NMI
model 1,0.260278,0.636757,0.296786,0.882979,0.739788


## Comparing Multiple Models

In [115]:
%run Functions/Functions.ipynb

def get_model_comparisons(models):
    # Create an empty DataFrame
    comparison_df = pd.DataFrame(columns=['Num Topics', 'Coherence (NPMI)', 'Silhouette Score', 'ARI', 'Purity', 'NMI'])
        
    # Most popular model - Hugging Face: https://huggingface.co/sentence-transformers?sort_models=downloads#models
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # remove stopwords/stop phrases containing 1 and 2 words
    # NGRAM_RANGE is an important choice
    stop_words = list(stopwords.words('english'))
    vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)


    for index, mod in enumerate(models):
        # Run custom model
        model = run_custom_BERTopic(docs, embedding_model, vectorizer_model, min_cluster_size=mod['min_cluster_size'], min_samples=mod['min_samples'])
        topics, probs = model.fit_transform(docs)
        
        topic_info_df = model.get_topic_info()
        
        num_topics = len(topic_info_df) - 1 # DOES NOT INCLUDE OUTLIERS TOPIC. ONLY COHERENT TOPICS

        # now get evaluation metrics

        ### Gather evaluation metrics
        coh = coherence_score(docs, model, vectorizer_model)
        
        # play with seeing if you actually need topics param here
        sil = silhouette_metric(docs, embedding_model, model, topics)
        
        true_values = df['Factor']
        predicted_values = model.get_document_info(docs)['Topic'] # NOTE THAT TOPIC -1 is included in the predicted_labels. Should we remove?!?!?!
        ari, purity, nmi = evaluation_metrics(true_values, predicted_values)
        
        comparison_df.loc["Model " + str(index) + " (min cluster: " + str(mod['min_cluster_size']) + ", min sample: " + str(mod['min_samples']) + ")"] = [
            num_topics, coh, sil, ari, purity, nmi]
        
    return comparison_df



# List of models as dictionaries
models = [
    {"min_cluster_size": 2, "min_samples": 1},
    {"min_cluster_size": 2, "min_samples": 2},
    {"min_cluster_size": 5, "min_samples": 5},
    {"min_cluster_size": 10, "min_samples": 10},
]

get_model_comparisons(models)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Num Topics,Coherence (NPMI),Silhouette Score,ARI,Purity,NMI
"Model 0 (min cluster: 2, min sample: 1)",33.0,0.260278,0.636757,0.296786,0.882979,0.739788
"Model 1 (min cluster: 2, min sample: 2)",13.0,0.170295,0.791262,0.680537,0.861702,0.836588
"Model 2 (min cluster: 5, min sample: 5)",8.0,0.061421,0.874702,0.787245,0.851064,0.874200
"Model 3 (min cluster: 10, min sample: 10)",6.0,-0.058653,0.783856,0.674083,0.723404,0.815289
